### OBTENCION DE DATOS API MERCADOLIBRE

#### Importacion de librerias necesarias

In [1]:
import pandas as pd
import requests
import json

#### Conociendo la API

El objetivo de esta seccion es obtener el listado completo de pokemons desde la API publica PokeApi (*https://pokeapi.co/*). Para este fin primero he leido la documentacion de la API para entender su funcionamiento y la forma en que se realizan los request ya que el resultado de la peticion varia dependediendo el paramentro enviado.<br>Una vez que definí los datos de interes de cada pokemon, he logrado determinar que para conseguir la informacion que deseo basta con dos request por pokemon. Entendiendo esto se define la funcion **get_data()** que realiza dos request por cada ID de pokemon y devuelve un diccionario con la respuesta de ambas solicitudes. Cada pokemon cuenta con un identificador y estos id están comprendidos en el rango del 1 al 1025.

In [5]:
def get_data(id:int) -> dict:
    data_pkm = {}

    #REQUEST 1
    url = f'https://pokeapi.co/api/v2/pokemon/{id}/'
    response = requests.get(url)
    if response.status_code == 200:
        data_pkm['Basic'] = response.json()
    else:
        raise Exception(f'Codigo de error: {response.status_code} en id: {id}')

    #REQUEST 2
    url = f'https://pokeapi.co/api/v2/pokemon-species/{id}/'
    response = requests.get(url)
    if response.status_code == 200:
        data_pkm['Plus'] = response.json()
    else:
        raise Exception(f'Codigo de error: {response.status_code} en id: {id}')
    
    return data_pkm

#### Obtencion de la informacion

En esta seccion obtendremos la informacion general de cada uno de los pokemons para ello se crea la lista **data_pkm** que se encarga de alojar los diccionarios resultantes de la funcion **get_data()** que posteriormente se ejecuta en un ciclo for con una cantidad de 1025 iteraciones.

In [57]:
data_pkm = []

for id in range(1,1026):
    data = get_data(id)
    data_pkm.append(data) 

print(f"La cantidad de elementos en la lista es: {len(data_pkm)}")

La cantidad de elementos en la lista es: 1025


Se guarda la informacion obtenida en un archivo tipo **JSON** de manera local

In [34]:
with open('.\Json\PokeApiData.json','w') as file:
    json.dump(data_pkm, file, indent=4, separators=(',',':'))

#### Extraccion de los datos de interes

Con el objetivo de extraer y estructurar los datos de interes que se obtubieron anteriormente se crea la clase **Pokemon()** quien se encarga de alojar mediante sus atributos los datos de interes. Tambien se crea la clase **PokemonBuilder()** la cual se encarga de extrer los datos de cada pokemon desde el diccionario que recibe por parametro, para luego devolver una instancia de la clase **Pokemon()** con sus atributos ya seteados.

In [58]:
class Pokemon():
    def __init__(self):
        self.id = None
        self.name = None
        self.evolves_from = None        
        self.generation = None        
        self.type1 = None
        self.type2 = None        
        self.hp = None
        self.atk = None
        self.Def = None
        self.spatk = None
        self.spdef = None
        self.speed = None
        self.total = None
        self.is_legenday = None
        self.is_mythical = None        
        self.abilities = None
        self.egg_groups = None
        self.varietes = None

class PokemonBuilder():
    def __init__(self, data : dict):
        self.data = data
        self.pkm = Pokemon()

    def build(self) -> Pokemon():
        self.pkm.id = self._get_id()
        self.pkm.name = self._get_name()
        self.pkm.is_legenday = self._is_legendary()
        self.pkm.is_mythical = self._is_mythical()
        self.pkm.generation = self._get_generation()
        self.pkm.evolves_from = self._get_evolve_from()
        self.pkm.type1 = self._get_type1()
        self.pkm.type2 = self._get_type2()
        self.pkm.egg_groups = self._get_egg_group()
        self.pkm.hp = self._get_hp_stat()
        self.pkm.atk = self._get_atk_stat()
        self.pkm.Def = self._get_def_stat()
        self.pkm.spatk = self._get_spatk_stat()
        self.pkm.spdef = self._get_spdef_stat()
        self.pkm.speed = self._get_spd_stat()
        self.pkm.total = self._get_total_stat()
        self.pkm.abilities = self._get_abilities()
        self.pkm.varietes = self._get_varietes()
        return self.pkm

    def _get_id(self):
        return self.data.get('Basic', None).get('id', None)
    
    def _get_name(self):
        return self.data.get('Basic', None).get('name', None)
    
    def _get_type1(self): 
        return self.data['Basic']['types'][0]['type']['name']
    
    def _get_type2(self):
        if len(self.data['Basic']['types']) > 1:
            return self.data['Basic']['types'][1]['type']['name']
        else:
            return None  

    def _get_hp_stat(self): 
        for item in self.data['Basic']['stats']:
            if item['stat']['name'] == 'hp':
                return item['base_stat']
        return None   
    
    def _get_atk_stat(self): 
        for item in self.data['Basic']['stats']:
            if item['stat']['name'] == 'attack':
                return item['base_stat']
        return None
    
    def _get_def_stat(self): 
        for item in self.data['Basic']['stats']:
            if item['stat']['name'] == 'defense':
                return item['base_stat']
        return None
    
    def _get_spatk_stat(self): 
        for item in self.data['Basic']['stats']:
            if item['stat']['name'] == 'special-attack':
                return item['base_stat']
        return None
    
    def _get_spdef_stat(self): 
        for item in self.data['Basic']['stats']:
            if item['stat']['name'] == 'special-defense':
                return item['base_stat']
        return None
    
    def _get_spd_stat(self): 
        for item in self.data['Basic']['stats']:
            if item['stat']['name'] == 'speed':
                return item['base_stat']
        return None
    
    def _get_total_stat(self):
        return self.pkm.hp + self.pkm.atk + self.pkm.Def + self.pkm.spatk + self.pkm.spdef + self.pkm.speed

    def _get_abilities(self):
        abilities = None
        for item in self.data['Basic']['abilities']:
            if abilities is None:
                abilities =item['ability']['name']
            else:
                abilities = abilities + '_' + item['ability']['name']
        return abilities
    
    def _is_legendary(self):
        return self.data['Plus']['is_legendary']
    
    def _is_mythical(self):
        return self.data['Plus']['is_mythical']
    
    def _get_generation(self):
        return self.data['Plus'].get('generation', {}).get('name', None)
    
    def _get_evolve_from(self):
        if self.data.get('Plus',{}).get('evolves_from_species', None) is not None:
            return self.data.get('Plus',{}).get('evolves_from_species', {}).get('name', None)
        else:
            return None
    
    def _get_egg_group(self):
        egg_groups = None
        for item in self.data['Plus']['egg_groups']:
            if egg_groups is None:            
                egg_groups = item['name']
            else:
                egg_groups = egg_groups + '_' + item['name']
        return egg_groups

    def _get_varietes(self):
        varietes = None
        if len(self.data['Plus']['varieties']) > 1:
            for item in self.data['Plus']['varieties']:
                if item['is_default'] == False: 
                    if varietes is None:
                        varietes = item['pokemon']['name']
                    else:
                        varietes = varietes + '_'+ item['pokemon']['name']
        return varietes

Se crea la lista **pokemons** con el objetivo de almacenar las instancias devueltas por la clase **PokemonBuilder()** cpn forma de diccionario. Se crea un ciclo for a fines de recorrer la lista **data_pkm** y por cada elemento en esta lista obtener una instancia de la clase **Pokemon()**

In [52]:
pokemons = []

for data in data_pkm:
    builder = PokemonBuilder(data)
    pkm = builder.build()
    pokemons.append(vars(pkm))

Se imprimen los datos en formato **JSON** para pre-visualizar el resultado obtenido 

In [59]:
print(json.dumps(pokemons, indent=4, separators=(',',':')))

[
    {
        "id":1,
        "name":"bulbasaur",
        "evolves_from":null,
        "generation":"generation-i",
        "type1":"grass",
        "type2":"poison",
        "hp":45,
        "atk":49,
        "Def":49,
        "spatk":65,
        "spdef":65,
        "speed":45,
        "total":318,
        "is_legenday":false,
        "is_mythical":false,
        "abilities":"overgrow_chlorophyll",
        "egg_groups":"monster_plant",
        "varietes":null
    },
    {
        "id":2,
        "name":"ivysaur",
        "evolves_from":"bulbasaur",
        "generation":"generation-i",
        "type1":"grass",
        "type2":"poison",
        "hp":60,
        "atk":62,
        "Def":63,
        "spatk":80,
        "spdef":80,
        "speed":60,
        "total":405,
        "is_legenday":false,
        "is_mythical":false,
        "abilities":"overgrow_chlorophyll",
        "egg_groups":"monster_plant",
        "varietes":null
    },
    {
        "id":3,
        "name":"venusaur",


Se guarda el resultado obtenido en un archivo **JSON** de manera local

In [53]:
with open('.\Json\PokemonData.json','w') as file:
    json.dump(pokemons, file, indent=4, separators=(',',':'))

Utilizando **Pandas** se puede leer la lista de diccionarios (**pokemons**) para obtener un **Dataframe** y manipular los datos segun la necesidad.

In [54]:
df = pd.DataFrame(pokemons)
df.head()

,id,name,evolves_from,generation,type1,type2,hp,atk,Def,spatk,spdef,speed,total,is_legenday,is_mythical,abilities,egg_groups,varietes
0,1,bulbasaur,None,generation-i,grass,poison,45,49,49,65,65,45,318,False,False,overgrow_chlorophyll,monster_plant,None
1,2,ivysaur,bulbasaur,generation-i,grass,poison,60,62,63,80,80,60,405,False,False,overgrow_chlorophyll,monster_plant,None
2,3,venusaur,ivysaur,generation-i,grass,poison,80,82,83,100,100,80,525,False,False,overgrow_chlorophyll,monster_plant,venusaur-mega_venusaur-gmax
3,4,charmander,None,generation-i,fire,None,39,52,43,60,50,65,309,False,False,blaze_solar-power,monster_dragon,None
4,5,charmeleon,charmander,generation-i,fire,None,58,64,58,80,65,80,405,False,False,blaze_solar-power,monster_dragon,None


Tambien se puede leer el archivo **JSON** creado anteriormente utilizando un **DataFrame** de **Pandas**

In [55]:
df = pd.read_json('.\Json\PokemonData.json')
df.head()

,id,name,evolves_from,generation,type1,type2,hp,atk,Def,spatk,spdef,speed,total,is_legenday,is_mythical,abilities,egg_groups,varietes
0,1,bulbasaur,None,generation-i,grass,poison,45,49,49,65,65,45,318,False,False,overgrow_chlorophyll,monster_plant,None
1,2,ivysaur,bulbasaur,generation-i,grass,poison,60,62,63,80,80,60,405,False,False,overgrow_chlorophyll,monster_plant,None
2,3,venusaur,ivysaur,generation-i,grass,poison,80,82,83,100,100,80,525,False,False,overgrow_chlorophyll,monster_plant,venusaur-mega_venusaur-gmax
3,4,charmander,None,generation-i,fire,None,39,52,43,60,50,65,309,False,False,blaze_solar-power,monster_dragon,None
4,5,charmeleon,charmander,generation-i,fire,None,58,64,58,80,65,80,405,False,False,blaze_solar-power,monster_dragon,None
